<a href="https://colab.research.google.com/github/casualcomputer/llm_google_colab/blob/main/llm_tracing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## LLM using Gemini (langchain)

In [ ]:
# Install dependencies
!pip install --upgrade pip --quiet
!pip install langchain google-generativeai torch torchvision transformers datasets langchain_community langchain_google_genai--quiet

ERROR: Could not find a version that satisfies the requirement langchain_google_genai--quiet (from versions: none)
ERROR: No matching distribution found for langchain_google_genai--quiet


In [1]:
# Imports & Authentication
import os
from datasets import load_dataset
import re
import pandas as pd
from google.colab import userdata

api_key = userdata.get('GOOGLE_API_KEY') # Use the name you gave your secret

if api_key:
    # Use the api_key
    print("API key loaded successfully from Colab Secrets.")
    # print(f"Your API key is: {api_key}") # Be careful printing the key itself
else:
    print("API key 'GOOGLE_API_KEY' not found in Colab Secrets.")

API key loaded successfully from Colab Secrets.


In [2]:
!pip install langchain_google_genai --quiet
import os
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
            model="gemini-1.5-flash-latest", # You can also try "gemini-pro"
            temperature=0,
            google_api_key=api_key
        )
print("LLM Initialized successfully.")

print("\nSending a test prompt to the LLM...")
print(type(llm))
print(llm.invoke("Write me a ballad about LangChain").content[:90])

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 25.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.
LLM Initialized successfully.

Sending a test prompt to the LLM...
<class 'langchain_google_genai.chat_models.ChatGoogleGenerativeAI'>
(Verse 1)
The data streams, a raging flood,
A chaos vast, misunderstood.
To find the gems,


# Agent

## Define Serper Tool

In [3]:
!pip install browserbase 'crewai[tools]' --quiet
from crewai_tools import SerperDevTool
import os
from google.colab import userdata

# Load the Serper API key from Colab Secrets
serper_api_key = userdata.get('SERPER_API_KEY') # Use the name you gave your Serper secret

if serper_api_key:
    # Set the SERPER_API_KEY environment variable
    os.environ["SERPER_API_KEY"] = serper_api_key
    print("SERPER_API_KEY loaded and set successfully from Colab Secrets.")
else:
    print("SERPER_API_KEY not found in Colab Secrets.")


# Initialize the tool for internet searching capabilities
# Now the tool will find the SERPER_API_KEY in the environment variables
Serpertool = SerperDevTool(
    search_url="https://google.serper.dev/scholar",
    n_results=10,
)

# You can now run the tool
#print(Serpertool.run(search_query="ChatGPT"))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.0/98.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 602.7/602.7 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.0/647.0 kB 27.6 MB/s eta

## Create a CrewAI agent

## Create prompt templates for agent switching

In [4]:
# 3. Define all your prompt templates from Section 1
ZERO_SHOT_TEMPLATE = "You are a helpful assistant. Answer concisely:\n\n{query}"
FEW_SHOT_TEMPLATE = (
    "You are a helpful assistant. Use the examples below to guide your answer.\n\n"
    "Example 1:\nQ: Who wrote 'Pride and Prejudice'?\nA: Jane Austen\n\n"
    "Example 2:\nQ: What is the boiling point of water in Celsius?\nA: 100°C\n\n"
    "Now answer:\nQ: {query}\nA:"
)
COT_TEMPLATE = (
    "You are a reasoning assistant. Think step by step:\n\n"
    "Question: {query}\n"
    "Answer: Let’s think step by step."
)
COT_SC_TEMPLATE = (
    "You are a reasoning assistant. Think carefully, step by step:\n\n"
    "Question: {query}\n"
    "Answer:"
)
REACT_TEMPLATE = (
    "You are a multi-agent reasoning assistant. Follow the ReAct format exactly:\n\n"
    "Thought: I need to gather facts for: {query}\n"
    "Action: Search[\"{query}\"]\n"
    "Observation:\n"
    "Thought: Now I have the necessary information.\n"
    "Answer:"
)
RAG_TEMPLATE = (
    "You have been provided the following context passages:\n\n{retrieved_docs}\n\n"
    "Using only the above context, answer:\n\n{query}"
)
CHAIN_STEP1 = (
    "Extract key facts needed to answer:\n\n"
    "Question: {query}\n"
    "Facts:"
)
CHAIN_STEP2 = (
    "You have these facts:\n\n{facts}\n\n"
    "Now answer:\n\n"
    "Question: {query}\n"
    "Answer:"
)
META_PROMPT = (
    "You are a prompt engineer. Given the user’s instruction, produce an optimized prompt:\n\n"
    "User instruction: {user_instruction}\n\n"
    "Optimized prompt:"
)
PAL_TEMPLATE = (
    "You are a Python interpreter. Write Python code that solves:\n\n"
    "{query}\n\n"
    "# After writing the code, execute it mentally and provide the result."
)

# 4. Build a dispatcher function to select the right template
def build_prompt(query: str, mode: str, **kwargs) -> str:
    if mode == "zero_shot":
        return ZERO_SHOT_TEMPLATE.format(query=query)
    elif mode == "few_shot":
        return FEW_SHOT_TEMPLATE.format(query=query)
    elif mode == "cot":
        return COT_TEMPLATE.format(query=query)
    elif mode == "cot_sc":
        return COT_SC_TEMPLATE.format(query=query)
    # elif mode == "react":
    #     return REACT_TEMPLATE.format(query=query)
    # elif mode == "rag":
    #     return RAG_TEMPLATE.format(retrieved_docs=kwargs.get("retrieved_docs", ""), query=query)
    elif mode == "chain_step1":
        return CHAIN_STEP1.format(query=query)
    elif mode == "chain_step2":
        return CHAIN_STEP2.format(query=query, facts=kwargs.get("facts", ""))
    elif mode == "meta":
        return META_PROMPT.format(user_instruction=query)
    elif mode == "pal":
        return PAL_TEMPLATE.format(query=query)
    else:
        raise ValueError(f"Unknown mode: {mode}")

# from langchain_core.messages import BaseMessage   # for the isinstance check

# def _to_text(resp) -> str:
#     """Normalise CrewAI responses to a clean string."""
#     if isinstance(resp, str):
#         return resp
#     if isinstance(resp, BaseMessage):
#         return resp.content
#     return str(resp)          # fallback

# def smart_llm_call(model, prompt):
#     """
#     Call the LLM using whichever entry-point the installed
#     CrewAI / LangChain version supports.
#     """
#     for m in ("invoke", "predict", "__call__"):
#         if hasattr(model, m):
#             resp = getattr(model, m)(prompt)     # call the method
#             return _to_text(resp)
#     raise AttributeError("LLM exposes no invoke/predict/__call__")

# # ------------------ MAIN DISPATCHER ------------------
# def call_with_mode(query: str, mode: str, **kwargs) -> str:
#     prompt = build_prompt(query, mode, **kwargs)
#     if mode == "react":          # needs the Search tool
#         return smart_llm_call(agent_react.llm, prompt).strip()
#     else:                        # every other prompting style
#         return smart_llm_call(llm, prompt).strip()

In [41]:
# # Repeating observations and not showing observations etc.
# # ── 1.  make a universal "stringify" helper ─────────────────────────
# def stringify_step(obj, max_obs_chars: int = 600) -> str:
#     """
#     Convert whatever CrewAI passes to step_callback into readable text.
#       • AgentAction   –> Thought / Action / Action Input
#       • ToolResult    –> Observation
#       • AgentFinish   –> Final Answer
#       • str           –> Final Answer (some builds)
#     Everything else falls back to repr(obj).
#     """

#     # -------- Final answer sometimes arrives as a bare string --------
#     if isinstance(obj, str):
#         return f"Final Answer: {obj.strip()}"

#     # -------- AgentAction -------------------------------------------
#     if hasattr(obj, "action") and hasattr(obj, "action_input"):
#         thought = getattr(obj, "log", "").strip()
#         return (
#             f"Thought: {thought}\n"
#             f"Action: {obj.action}\n"
#             f"Action Input: {obj.action_input}"
#         )

#     # -------- ToolResult  (Observation) -----------------------------
#     if hasattr(obj, "result") or hasattr(obj, "output"):
#         observation = getattr(obj, "result", getattr(obj, "output", ""))
#         # shorten very long JSON blobs so your log stays readable
#         obs_text = (
#             observation if len(str(observation)) <= max_obs_chars
#             else str(observation)[:max_obs_chars] + " …[truncated]…"
#         )
#         return f"Observation: {obs_text}"

#     # -------- AgentFinish -------------------------------------------
#     if hasattr(obj, "return_values"):
#         final_answer = obj.return_values.get("output", "").strip()
#         return f"Final Answer: {final_answer}"

#     # -------- Fallback ----------------------------------------------
#     return repr(obj)

# def stringify_step(obj, max_obs_chars: int = 600) -> str:
#     # Final answer sometimes arrives as bare str
#     if isinstance(obj, str):
#         return f"Final Answer: {obj.strip()}"

#     # AgentAction (Thought / Action / Action Input)
#     if hasattr(obj, "action") and hasattr(obj, "action_input"):
#         return (
#             f"Thought: {getattr(obj, 'log', '').strip()}\n"
#             f"Action: {obj.action}\n"
#             f"Action Input: {obj.action_input}"
#         )

#     # ToolResult – prepend the *previous* Thought/Action in .log
#     if hasattr(obj, "result") or hasattr(obj, "output"):
#         prior_log = getattr(obj, "log", "").strip()          # ← new!
#         obs = getattr(obj, "result", getattr(obj, "output", ""))
#         if len(str(obs)) > max_obs_chars:
#             obs = str(obs)[:max_obs_chars] + " …[truncated]…"
#         return f"{prior_log}\nObservation: {obs}"

#     # AgentFinish
#     if hasattr(obj, "return_values"):
#         return f"Final Answer: {obj.return_values.get('output', '').strip()}"

#     return repr(obj)

# 0. a list to hold them
raw_steps: list = []

def collect_raw(obj, **_):
    """
    CrewAI passes AgentAction / ToolResult / AgentFinish
    (or sometimes a plain str).  Keep each object intact.
    """
    import copy                     # avoid later mutation
    raw_steps.append(copy.deepcopy(obj))


In [30]:
import inspect, crewai, importlib.metadata
print(importlib.metadata.version("crewai"))
print("Has AgentAction?", 'AgentAction' in dir(__import__('crewai.agents.parser').agents.parser))


0.121.1
Has AgentAction? True


In [43]:
!pip install crewai crewai_tools --quiet
from crewai import LLM, Agent, Task, Crew
import os
import io, sys, contextlib

# ---- 1. LLM wrapper -------------------------------------------------
llm_crew = LLM(
    model="gemini/gemini-1.5-flash-latest",       # provider prefix
    api_key=api_key          # set GOOGLE_API_KEY env var
)


# ---- 3. ReAct agent --------------------------------------------------
agent_react = Agent(
    role="Factual Researcher",
    goal="Find accurate facts quickly",
    backstory=(
        "You are a detail-oriented research analyst who uses web search "
        "to answer geography and demographic questions with concise, up-to-date data."
    ),
    llm=llm_crew,
    tools=[Serpertool],
    step_callback= collect_raw, # collect_step,   # register our function to receive every step
    verbose=True          # prints Thought / Action / Observation
)

# ---- 4. One-off task runner -----------------------------------------
!pip install rich
from crewai import Task, Crew    # already imported Agent earlier
from rich import get_console

def run_single_agent(query: str) -> str:
    #step_trace.clear()              # reset collector
    task = Task(
        description=query,
        expected_output="Concise answer",
        agent=agent_react
    )
    crew = Crew(agents=[agent_react], tasks=[task])
    #result = crew.kickoff() #working
    #result = crew.kickoff_for_each(inputs=[{"input": query}])#[0]

    # 1. Rich logging didn't work
    # console = get_console()             # the console Rich already uses
    # with console.capture() as cap:      # start recording *before* kickoff
    #     result = crew.kickoff_for_each(inputs=[{"input": query}])[0].raw
    # rich_log = cap.get()                # full coloured (→ plain) transcript

    # 2. This didn't work
    buf = io.StringIO()
    with contextlib.redirect_stdout(buf), contextlib.redirect_stderr(buf):
        # Everything CrewAI prints (via Rich or plain prints) goes into buf
        result = crew.kickoff_for_each(inputs=[{"input": query}])[0].raw

    # 4) Grab the full transcript
    rich_log = buf.getvalue()

    # Normalise return type
    if isinstance(result, str):
        return result.strip(), rich_log
    if isinstance(result, dict):
        return result, rich_log
    return str(result).strip(), rich_log

## Make LLM calls

In [40]:
step_trace

['\nObservation: {\'searchParameters\': {\'q\': \'capital of Canada and population\', \'type\': \'search\', \'num\': 10, \'engine\': \'google\'}, \'organic\': [{\'title\': \'Canada - Wikipedia\', \'link\': \'https://en.wikipedia.org/wiki/Canada\', \'snippet\': "Canada\'s capital is Ottawa and its three largest metropolitan areas are Toronto, Montreal, and Vancouver.", \'position\': 1, \'sitelinks\': [{\'title\': \'Largest population centres\', \'link\': \'https://en.wikipedia.org/wiki/List_of_the_largest_population_centres_in_Canada\'}, {\'title\': \'Otta …[truncated]…',
 '\nObservation: {\'searchParameters\': {\'q\': \'capital of Canada and population\', \'type\': \'search\', \'num\': 10, \'engine\': \'google\'}, \'organic\': [{\'title\': \'Canada - Wikipedia\', \'link\': \'https://en.wikipedia.org/wiki/Canada\', \'snippet\': "Canada\'s capital is Ottawa and its three largest metropolitan areas are Toronto, Montreal, and Vancouver.", \'position\': 1, \'sitelinks\': [{\'title\': \'Larg

In [45]:
raw_steps

[ToolResult(result="{'searchParameters': {'q': 'population of Ottawa Canada', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Ottawa - Wikipedia', 'link': 'https://en.wikipedia.org/wiki/Ottawa', 'snippet': 'As of 2021, Ottawa had a city population of 1,017,449 and a metropolitan population of 1,488,307, making it the fourth-largest city and fourth-largest ...', 'position': 1, 'sitelinks': [{'title': 'Demographics', 'link': 'https://en.wikipedia.org/wiki/Demographics_of_Ottawa'}, {'title': 'History', 'link': 'https://en.wikipedia.org/wiki/History_of_Ottawa'}, {'title': 'List of neighbourhoods in...', 'link': 'https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Ottawa'}, {'title': 'Timeline of Ottawa history', 'link': 'https://en.wikipedia.org/wiki/Timeline_of_Ottawa_history'}]}, {'title': 'Demographics of Ottawa - Wikipedia', 'link': 'https://en.wikipedia.org/wiki/Demographics_of_Ottawa', 'snippet': 'In 2021, the population of the city of Ottawa was 1,017,44

In [53]:
triple = raw_steps      # the list you showed
print(raw_steps)

tool_res   = triple[0]          # ToolResult
action_obj = triple[1]          # AgentAction
finish_obj = triple[2]          # AgentFinish

print(dir(action_obj))        # or:  pprint(vars(action_obj))

# 1) Print the LLM’s “thought” string:
print("Thought:\n", action_obj.thought)

# 2) Print which tool was invoked (e.g. "Search"):
print("Action :", action_obj.tool)

# 3) Print the payload sent to that tool:
print("Input  :", action_obj.tool_input)

print("Action via tool use:", tool_res)

print("Final Answer:\n", finish_obj.text)

[ToolResult(result="{'searchParameters': {'q': 'population of Ottawa Canada', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Ottawa - Wikipedia', 'link': 'https://en.wikipedia.org/wiki/Ottawa', 'snippet': 'As of 2021, Ottawa had a city population of 1,017,449 and a metropolitan population of 1,488,307, making it the fourth-largest city and fourth-largest ...', 'position': 1, 'sitelinks': [{'title': 'Demographics', 'link': 'https://en.wikipedia.org/wiki/Demographics_of_Ottawa'}, {'title': 'History', 'link': 'https://en.wikipedia.org/wiki/History_of_Ottawa'}, {'title': 'List of neighbourhoods in...', 'link': 'https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Ottawa'}, {'title': 'Timeline of Ottawa history', 'link': 'https://en.wikipedia.org/wiki/Timeline_of_Ottawa_history'}]}, {'title': 'Demographics of Ottawa - Wikipedia', 'link': 'https://en.wikipedia.org/wiki/Demographics_of_Ottawa', 'snippet': 'In 2021, the population of the city of Ottawa was 1,017,44

In [54]:
# 1) Re‐use a “stringify” helper that knows how to format each step
def stringify_step(obj, max_obs_chars: int = 600) -> str:
    # If it arrives as a bare string, that’s the final answer
    if isinstance(obj, str):
        return f"Final Answer: {obj.strip()}"
    # If it’s an AgentAction, print thought / tool name / tool_input
    if hasattr(obj, "thought") and hasattr(obj, "tool_input"):
        return (
            f"Thought: {obj.thought.strip()}\n"
            f"Action: {obj.tool}\n"
            f"Action Input: {obj.tool_input}"
        )
    # If it’s a ToolResult, include the preceding thought (in .text or .log) and the result
    # (some versions store the prior thought in .text or .log, adjust accordingly)
    if hasattr(obj, "result"):
        # many builds put the prior “thought” text into obj.text
        prior = getattr(obj, "text", "").strip() or ""
        obs   = obj.result
        if len(str(obs)) > max_obs_chars:
            obs = str(obs)[:max_obs_chars] + " …[truncated]…"
        if prior:
            return f"{prior}\nObservation: {obs}"
        else:
            return f"Observation: {obs}"
    # If it’s an AgentFinish, the final answer lives in obj.text
    if hasattr(obj, "text"):
        return f"Final Answer: {obj.text.strip()}"
    # Fallback to repr()
    return repr(obj)

# 2) Convert every raw step into a string
text_lines = [stringify_step(o) for o in raw_steps]

# 3) Now either print them or write to disk. For example:
with open("trace_log.txt", "w", encoding="utf-8") as f:
    for line in text_lines:
        f.write(line.replace("\n", "\n    ") + "\n\n")  # indent continuations

# 4) If you just want a single big string:
full_text = "\n\n".join(text_lines)
print(full_text)


Observation: {'searchParameters': {'q': 'population of Ottawa Canada', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Ottawa - Wikipedia', 'link': 'https://en.wikipedia.org/wiki/Ottawa', 'snippet': 'As of 2021, Ottawa had a city population of 1,017,449 and a metropolitan population of 1,488,307, making it the fourth-largest city and fourth-largest ...', 'position': 1, 'sitelinks': [{'title': 'Demographics', 'link': 'https://en.wikipedia.org/wiki/Demographics_of_Ottawa'}, {'title': 'History', 'link': 'https://en.wikipedia.org/wiki/History_of_Ottawa'}, {'title': 'List of neighbourho …[truncated]…

Thought: tool_code
Thought: I need to find the capital city of Canada and its population.  I will use a search engine to find this information.
Action: Search the internet with Serper
Action Input: {"search_query": "population of Ottawa Canada"}

Final Answer: Thought: I now know the final answer
Final Answer: The capital of Canada is Ottawa.  As of 2021, Ottawa had a 

In [44]:
from langchain_core.messages import BaseMessage   # for the isinstance check

def _to_text(resp) -> str:
    """Normalise CrewAI responses to a clean string."""
    if isinstance(resp, str):
        return resp
    if isinstance(resp, BaseMessage):
        return resp.content
    return str(resp)          # fallback

def smart_llm_call(model, prompt):
    """
    Call the LLM using whichever entry-point the installed
    CrewAI / LangChain version supports.
    """
    for m in ("invoke", "predict", "__call__"):
        if hasattr(model, m):
            resp = getattr(model, m)(prompt)     # call the method
            return _to_text(resp)

# ------------------ MAIN DISPATCHER ------------------
def call_with_mode(query: str, mode: str, **kwargs) -> str:

    # if mode == "react":          # needs the Search tool
    #   return smart_llm_call(agent_react.llm, prompt).strip()
    if mode == "react":
      return run_single_agent(query)
    else:                        # every other prompting style
      prompt = build_prompt(query, mode, **kwargs)
      return smart_llm_call(llm, prompt).strip()

# 6. Example Calls
query_example = "What is the capital of Canada and its population?"
react_answer, react_log = call_with_mode(query_example, "react")
print("react →", react_answer)
print("react log →", react_log)

# 6A. Zero-Shot
res_zero = call_with_mode(query_example, mode="zero_shot")
print("Zero-Shot:", res_zero)  # :contentReference[oaicite:24]{index=24}
print("...................")

# 6B. Few-Shot
res_few = call_with_mode(query_example, mode="few_shot")
print("Few-Shot:", res_few)    # :contentReference[oaicite:25]{index=25}
print("...................")

# 6C. Chain-of-Thought (CoT)
res_cot = call_with_mode(query_example, mode="cot")
print("CoT:", res_cot)         # :contentReference[oaicite:26]{index=26}
print("...................")

# 6D. Self-Consistency (CoT-SC) (voting over 3 runs)
cot_sc_chains = [call_with_mode(query_example, mode="cot_sc") for _ in range(3)]
finals = [chain.split("\n")[-1].strip() for chain in cot_sc_chains if "Answer:" in chain]
from collections import Counter
res_cot_sc = Counter(finals).most_common(1)[0][0] if finals else ""
print("CoT-SC Vote:", res_cot_sc)  # :contentReference[oaicite:27]{index=27}
print("...................")

# # 6E. ReAct (tool call)
# res_react = call_with_mode(query_example, mode="react")
# print("ReAct:", res_react)     # :contentReference[oaicite:28]{index=28}

# 6G. Prompt Chaining (Two-Step)
facts = call_with_mode(query_example, mode="chain_step1")
print("Chained Facts:", facts)  # :contentReference[oaicite:30]{index=30}
res_chain_final = call_with_mode(query_example, mode="chain_step2", facts=facts)
print("Chained Final:", res_chain_final)  # :contentReference[oaicite:31]{index=31}
print("...................")

# 6H. Meta Prompting
opt_prompt = call_with_mode("Optimize a prompt for: " + query_example, mode="meta")
print("Meta Optimized Prompt:", opt_prompt)  # :contentReference[oaicite:32]{index=32}
print("...................")

# 6I. Program-Aided Language Model (PAL)
res_pal = call_with_mode(query_example, mode="pal")
print("PAL:", res_pal)         #
print("...................")

# # 6F. Retrieval-Augmented Generation (RAG)
# # Assume retrieve_docs is implemented elsewhere
# def retrieve_docs(q: str, k: int = 3) -> str:
#     # E.g., use a FAISS vector store or Wikipedia API
#     return "Ottawa is the capital of Canada. The population is ~1 million."

# docs = retrieve_docs(query_example, k=3)
# res_rag = call_with_mode(query_example, mode="rag", retrieved_docs=docs)
# print("RAG:", res_rag)         # :contentReference[oaicite:29]{index=29}

react → The capital of Canada is Ottawa.  As of 2021, Ottawa had a city population of 1,017,449.
react log → 


In [ ]:
query = "What is the capital of Ottawa and its population?"
for mode in ["zero_shot", "few_shot", "cot", "cot_sc"]:r
    print(f"{mode:<8} →", call_with_mode(query, mode))

zero_shot → Canberra; approximately 430,000
few_shot → A: The capital of Australia is Canberra.  Its population is approximately 430,000.
cot      → Let’s think step by step.

Step 1: Identify the country in question. The question asks about the capital of Australia.

Step 2: Determine the capital city. The capital city of Australia is Canberra.

Step 3: Find the population of the capital city.  A quick online search reveals that the population of Canberra is approximately 430,000 (this number fluctuates and depends on the source and year).

Step 4: Combine the findings.  Therefore, the capital of Australia is Canberra, and its population is approximately 430,000.
cot_sc   → Step 1: Identify the question. The question asks for the capital city of Australia and its population.

Step 2: Determine the capital city. The capital city of Australia is Canberra.

Step 3: Find the population of Canberra.  This requires looking up the population data from a reliable source (like the Australian B

# Model Switching Demo

# Agent: Factual Researcher
## Task: What is the capital of Australia and its population?











# Agent: Factual Researcher
## Thought: tool_code
Thought: I need to find the capital of Australia and its population using a web search.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"capital of Australia and population\"}"
## Tool Output: 
{'searchParameters': {'q': 'capital of Australia and population', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Canberra - Wikipedia', 'link': 'https://en.wikipedia.org/wiki/Canberra', 'snippet': "As of June 2024, Canberra's estimated population was 473,855. Canberra. Kanbarra (Ngunawal). Australian Capital ...", 'position': 1, 'sitelinks': [{'title': 'History of Canberra', 'link': 'https://en.wikipedia.org/wiki/History_of_Canberra'}, {'title': 'Demographics of Canberra', 'link': 'https://en.wikipedia.org/wiki/Demographics_of_Canberra'}, {'title': 'Canberra (disambiguation)', 'link': '